# Installings

In [1]:
pip install requests opencv-python numpy pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Baixando as imagens

In [2]:
import pandas as pd
import requests
import cv2
import unittest
from PIL import Image
from io import BytesIO
import os

Criei uma função pra baixar imagens e salvar os arquivos localmente

In [3]:
urls = [
    'https://cdn.pixabay.com/photo/2017/02/20/18/03/cat-2083492_960_720.jpg',
    'https://cdn.pixabay.com/photo/2015/04/23/22/00/tree-736885_960_720.jpg',
    'https://cdn.pixabay.com/photo/2015/06/19/21/24/avenue-815297_960_720.jpg',
    'https://cdn.pixabay.com/photo/2013/10/02/23/03/mountains-190055_960_720.jpg',
    'https://cdn.pixabay.com/photo/2024/04/26/14/52/nuthatch-8722044_1280.jpg',
    'https://cdn.pixabay.com/photo/2014/12/16/22/25/sunset-570881_960_720.jpg',
    'https://cdn.pixabay.com/photo/2024/04/25/11/55/ai-generated-8719633_960_720.png',
    'https://cdn.pixabay.com/photo/2015/12/01/20/28/road-1072823_960_720.jpg',
    'https://cdn.pixabay.com/photo/2017/12/10/15/16/white-horse-3010129_960_720.jpg',
    'https://cdn.pixabay.com/photo/2024/04/28/05/07/bird-8724812_960_720.jpg'
]

In [4]:
def criar_diretorio_se_necessario(diretorio):
    """Cria um diretório se ele não existir."""
    if not os.path.exists(diretorio):
        os.makedirs(diretorio)
        print(f"Diretório '{diretorio}' criado.")
    else:
        print(f"Diretório '{diretorio}' já existe.")

def baixar_imagem(url, diretorio, nome_arquivo):
    """Baixa uma imagem da URL especificada e a salva em um diretório."""
    try:
        response = requests.get(url)
        response.raise_for_status() 
        imagem = Image.open(BytesIO(response.content))
        caminho_completo = os.path.join(diretorio, nome_arquivo)
        imagem.save(caminho_completo)
        print(f"Imagem salva em: {caminho_completo}")
        return True
    except requests.RequestException as e:
        print(f"Erro ao baixar a imagem de {url}: {e}")
        return False
    except IOError as e:
        print(f"Erro ao salvar a imagem em {caminho_completo}: {e}")
        return False

# Exemplo de uso:
diretorio = os.getcwd() + './images'
criar_diretorio_se_necessario(diretorio)
for i, url in enumerate(urls):
    baixar_imagem(url, diretorio, f"image_{i}.jpg")


Diretório 'c:\Users\cazem\OneDrive\Documentos\Github\INTELI-activities\m10\semana_2./images' criado.
Imagem salva em: c:\Users\cazem\OneDrive\Documentos\Github\INTELI-activities\m10\semana_2./images\image_0.jpg
Imagem salva em: c:\Users\cazem\OneDrive\Documentos\Github\INTELI-activities\m10\semana_2./images\image_1.jpg
Imagem salva em: c:\Users\cazem\OneDrive\Documentos\Github\INTELI-activities\m10\semana_2./images\image_2.jpg
Imagem salva em: c:\Users\cazem\OneDrive\Documentos\Github\INTELI-activities\m10\semana_2./images\image_3.jpg
Imagem salva em: c:\Users\cazem\OneDrive\Documentos\Github\INTELI-activities\m10\semana_2./images\image_4.jpg
Imagem salva em: c:\Users\cazem\OneDrive\Documentos\Github\INTELI-activities\m10\semana_2./images\image_5.jpg
Imagem salva em: c:\Users\cazem\OneDrive\Documentos\Github\INTELI-activities\m10\semana_2./images\image_6.jpg
Imagem salva em: c:\Users\cazem\OneDrive\Documentos\Github\INTELI-activities\m10\semana_2./images\image_7.jpg
Imagem salva em: c:

## Processamento

### Convertendo a escala de cinza

In [5]:
def converter_para_cinza(imagem_path):
    imagem = cv2.imread(imagem_path)
    return cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY) if imagem is not None else None


### Redimensionamento de imagens

In [6]:
def redimensionar(imagem, escala=0.5):
    altura = int(imagem.shape[0] * escala)
    largura = int(imagem.shape[1] * escala)
    dimensoes = (largura, altura)
    return cv2.resize(imagem, dimensoes, interpolation=cv2.INTER_AREA) if imagem is not None else None

### Detectando bordas

In [7]:
def detectar_bordas(imagem):
    return cv2.Canny(imagem, 100, 200) if imagem is not None else None

### Aumento de contraste

In [8]:
def aumentar_contraste(imagem):
    if imagem is None:
        return None
    lab = cv2.cvtColor(imagem, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge([cl, a, b])
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

### Gaussian Blur

In [9]:
def aplicar_gaussian_blur(imagem):
    return cv2.GaussianBlur(imagem, (5, 5), 0) if imagem is not None else None

### Rotacionando

In [10]:
def rotacionar_imagem(imagem, angulo=90):
    altura, largura = imagem.shape[:2]
    ponto_central = (largura//2, altura//2)
    matriz_rotacao = cv2.getRotationMatrix2D(ponto_central, angulo, 1.0)
    return cv2.warpAffine(imagem, matriz_rotacao, (largura, altura)) if imagem is not None else None

## Testes e Visualização

In [11]:
dados = []
image_paths = ['images/image_{}.jpg'.format(i) for i in range(len(urls))]
for url, path in zip(urls, image_paths):
  img_cinza = converter_para_cinza(path)
  img_redimensionada = redimensionar(img_cinza)
  img_bordas = detectar_bordas(img_redimensionada)
  img_original = cv2.imread(path)
  img_contraste = aumentar_contraste(img_original)
  img_blur = aplicar_gaussian_blur(img_contraste)
  img_rotacionada = rotacionar_imagem(img_original)
  dados.append([path, img_cinza, img_redimensionada, img_bordas, img_contraste, img_blur, img_rotacionada])

In [12]:
df = pd.DataFrame(dados, columns=['Path', 'Cinza', 'Redimensionada', 'Bordas', 'Contraste', 'Blur', 'Rotacionada'])

### Testes

In [13]:
class TesteProcessamento(unittest.TestCase):
    def test_redimensionamento(self):
        for _, row in df.iterrows():
            self.assertIsNotNone(row['Redimensionada'], "Imagem redimensionada está vazia")

### Visualização

In [14]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

df.head()

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


,Path,Cinza,Redimensionada,Bordas,Contraste,Blur,Rotacionada
0,images/image_0.jpg,"[[33, 51, 70, 55, 32, 41, 41, 10, 18, 26, 29, ...","[[32, 52, 32, 28, 18, 25, 32, 33, 29, 34, 43, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[[32, 63, 54], [64, 100, 89], [89, 125, 117],...","[[[32, 60, 50], [39, 68, 58], [49, 80, 70], [4...","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
1,images/image_1.jpg,"[[12, 13, 14, 16, 18, 20, 22, 23, 30, 32, 34, ...","[[13, 15, 19, 21, 29, 34, 34, 31, 26, 29, 25, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[[24, 15, 9], [27, 15, 11], [28, 19, 17], [34...","[[[26, 16, 11], [27, 16, 13], [29, 19, 16], [3...","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
2,images/image_2.jpg,"[[8, 29, 12, 15, 60, 47, 83, 83, 84, 35, 11, 1...","[[15, 27, 51, 118, 67, 13, 14, 17, 16, 33, 13,...","[[255, 255, 255, 0, 255, 0, 0, 0, 0, 0, 0, 0, ...","[[[19, 27, 18], [65, 95, 72], [27, 40, 23], [3...","[[[45, 59, 44], [46, 61, 46], [55, 75, 58], [7...","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
3,images/image_3.jpg,"[[24, 25, 26, 27, 28, 28, 28, 28, 27, 26, 25, ...","[[25, 27, 28, 28, 27, 25, 24, 25, 25, 25, 25, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[[33, 21, 10], [36, 24, 14], [38, 28, 14], [4...","[[[38, 27, 15], [38, 27, 16], [40, 28, 18], [4...","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
4,images/image_4.jpg,"[[90, 89, 89, 88, 88, 89, 89, 90, 89, 89, 89, ...","[[89, 88, 88, 89, 87, 88, 87, 85, 86, 86, 86, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[[75, 91, 126], [71, 88, 121], [71, 88, 121],...","[[[68, 85, 119], [67, 85, 118], [66, 84, 116],...","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
